![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [115]:
#Import necessary libraries and create DataFrame
import pandas as pd
ds_jobs = pd.read_csv("customer_train.csv")
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [116]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

In [117]:
#EDA
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [118]:
for col in ds_jobs_transformed.columns:
    if ds_jobs_transformed[col].dtypes == "object":
        print(ds_jobs_transformed[col].value_counts(), "\n")

city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64 

Male      13221
Female     1238
Other       191
Name: gender, dtype: int64 

Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64 

no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64 

Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64 

STEM               14492
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: major_discipline, dtype: int64 

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
1

In [119]:
#Create dictionaries for boolean and ordinal categories
boolean_categories = {
    "relevant_experience": {"Has relevant experience": True, "No relevant experience": False},
    "job_change": {0.0: False, 1.0: True}}
ordinal_categories = {
    "enrolled_university": ["no_enrollment", "Part time course", "Full time course"],
    "education_level": ["Primary School", "High School", "Graduate", "Masters", "Phd"],
    "experience": ["<1"] + list(map(str, range(1, 21))) + [">20"],
    "company_size": ["<10", "10-49", "50-99", "100-499", "500-999", "1000-4999", "5000-9999", "10000+"],
    "last_new_job": ["never", "1", "2", "3", "4", ">4"]}

In [120]:
#Use loop for type conversion
for col in ds_jobs_transformed.columns:
    if col in ["relevant_experience", "job_change"]:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].map(boolean_categories[col])
    elif col in ["student_id", "training_hours"]:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype("int32")
    elif col in ["city_development_index"]:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype("float16")
    elif col in ["city", "gender", "major_discipline", "company_type"]:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype("category")
    elif col in ordinal_categories.keys():
        category = pd.CategoricalDtype(categories= ordinal_categories[col], ordered=True)
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype(category)

In [121]:
ds_jobs_transformed.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object

In [122]:
#filtering DataFrame
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed["experience"] >= "10") & (ds_jobs_transformed["company_size"] >= "1000-4999")]